### Bronze Layer: Ingest constructors.json

Load raw JSON from landing zone and save as parquet in bronze/raw layer.

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import current_timestamp, lit

In [ ]:
from formula1.formula1_constants import landing_folder_path, raw_folder_path

##### Step 1 - Define schema and read JSON from landing zone

In [ ]:
constructors_schema = StructType(fields=[
    StructField("constructorId", IntegerType(), False),
    StructField("constructorRef", StringType(), True),
    StructField("name", StringType(), True),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True)
])

In [ ]:
constructors_df = spark.read \
    .schema(constructors_schema) \
    .json(f"{landing_folder_path}/constructors.json")

##### Step 2 - Add ingestion metadata

In [ ]:
constructors_df = constructors_df \
    .withColumn("ingestion_date", current_timestamp()) \
    .withColumn("data_source", lit(v_data_source))

##### Step 3 - Write to bronze/raw layer as parquet

In [ ]:
constructors_df.write.mode("overwrite").parquet(f"{raw_folder_path}/constructors")